<a href="https://colab.research.google.com/github/colinmcnamara/Digital_Deming/blob/main/Digital_Deming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up your local environment 

On intial run of "import colab_env" it will create vars.env in the root of your Google Drive

COLAB_ENV = Active

export OPENAI_API_KEY = "your_key"

export SERPAPI_API_KEY = "your_key"


Install my local packages

In [1]:
pip install OpenAI langchain pydantic faiss-gpu colab-env unstructured chromadb tiktoken pdf2image faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 77.9 

Yo, let's import our local env so we don't have to show the world our keys

In [2]:
import colab_env

Mounted at /content/gdrive


Let's import our python modules

In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import OpenAI, VectorDBQA
from langchain.chains import RetrievalQA
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.document_loaders import DirectoryLoader
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
import magic
import os
import nltk
import textwrap



Let's set the temperature of our AI models

In [4]:
llm = OpenAI(temperature=0)
chat = ChatOpenAI(temperature=0)

Let's specify the directory and file type for our Demming source docs

In [5]:
loader = DirectoryLoader('/content/gdrive/MyDrive/Projects/Digital_Demming/src_content', glob='**/*.pdf')

Load our documents


In [6]:
documents = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Check how many documents are loaded

In [7]:
len (documents)

1

Let's split our document up into chunks

In [8]:
#text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap  = 20,
    length_function = len,
    separators=["\n\n", "\n", " ", ""]
)

Let' create a variable named texts, and split these documents into it

In [9]:
texts = text_splitter.split_documents(documents)

Let's tell it that we want it to use OpenAI embeddings, and tell it what env variable the key is held

In [10]:
embeddings = OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY'])

Let's create a variable named docsearch, and pull from Chroma vectorstore including texts, and the embeddings)

In [11]:
docsearch = Chroma.from_documents(texts, embeddings)

define qa chain, which pulls from VectorDBQA, and use the LLM chain named OpenAI. We specify that it "Stuff"'s all the data in. We could use "map_reduce" instead to optimize for performance and scale. Let's tell it to use the vectorstore we defined earlier named docsearch

note, i'm trying RetrievalQA because VectorDBQA is depricated

In [32]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch, return_source_documents=True)
query = "Describe in depth, what is the foundation of quality. Provide three examples using bullet points"
result = qa({"query": query})

In [13]:
result['result']

'\n\nThe foundation of quality involves establishing processes, systems, and standards that guarantee the consistency and reliability of products and services. Examples include:\n \n• Establishing processes and systems to ensure that product requirements are met consistently, such as quality control systems, quality assurance procedures, and customer feedback systems.\n• Establishing standards to measure the quality of products and services, such as the American Standards Association (ASA), British Standards (BS), and European Organization for Quality (EOQ).\n• Establishing quality management organizations, such as the Advisory Group on Reliability of Electronic Equipment (AGREE), Association for Overseas Technical Scholarship (AOTS), Asian Productivity Organization (APO), American Society for Quality Control (ASQC), Civil Communication Section (CCS), Control System Committee (COSCO), European Organization for Quality Control (EOQC), and European Foundation for Quality Management (EFQM

In [14]:
result['source_documents']

[Document(page_content='1) AGREE: Advisory Group on Reliability of Electronic Equipment 2) AOTS: Association for Overseas Technical Scholarship 3) APO: Asian Productivity Organization 4) ASA: American Standards Association 5) ASQC: American Society for Quality Control 6) BS: British Standards 7) CCS: Civil Communication Section 8) COSCO: Control System Committee 9) EOQC: European Organization for Quality Control 10) EOQ: European Organization for Quality 11) EFQM: European Foundation for Quality Management 12) FMES: Japan Federation of Managerial Engineering Societies\n\n524\n\nA Parallel Chronology of Prof. Kaoru Ishikawa’s Life and TQC in Japan', metadata={'source': '/content/gdrive/MyDrive/Projects/Digital_Demming/src_content/ParallelChronology_Ver2_150717.pdf'}),
 Document(page_content='Major Events in the Development of QC\n\nCourse. (July–Dec. 1957, JUSE)\n\n\uf09f The Chugoku Quality Control Research Group was established. (JSA Hiroshima) \uf09f The Design of Experiment Seminar 

In [15]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch, return_source_documents=True)
#query = "what is the impact of JUSE on TQM?"
#query = "what is the purpouse of life?"
#query = "what does demming like for lunch"
#query = "is quality like a human"
#query = "what is the fourth rule of TQM"
#query = "what is the most important concept in TQM"
#query =  "WHo won the deming prize?"
#query = "Is TQM like lean?"
#query = "How are TQM and Lean different?"
query = "What is lean?"

result = qa({"query": query})

In [16]:
result['result']

" I don't know."

In [17]:
result['source_documents']

[Document(page_content='held. (July, JUSE)\n\n\uf09f Governmental Building Construction Q-S Round-table conference was held. (JSA)\n\n\uf09f The Mexican Quality Control Training\n\n(Age 62)\n\n515\n\nA Parallel Chronology of Prof. Kaoru Ishikawa’s Life and TQC in Japan\n\nYear\n\nProf. Ishikawa’s Life\n\nMajor Events in the Development of QC\n\n\uf09f He became a researcher at the\n\nTeam visited Japan. (JICA/JUSE)\n\nComponent Measurement Department, Laboratory of Metrology, the Agency of Industrial Science and Technology. \uf09f He became the chair of the executive\n\ncommittee of the 20th National Meeting of Standardization. (JSA)\n\n\uf09f He became the chair of the organization committee of the Mexican Quality Control Training Course. (JICA/JUSE)\n\n\uf09f He was awarded with the Medal with\n\nBlue Ribbon for his contribution to the promotion of industrial standardization. \uf09f He became an honorary member of the Argentine Association for Quality Control.\n\n\uf09f The 1st PL Wo

# The History and Evolution of Total Quality Management (TQM): The Contributions of Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)

## I. Introduction
- Definition of Total Quality Management (TQM)
- Brief introduction of the significance of TQM in the modern business world
- Introduction to Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)
- Thesis Statement

In [18]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch, return_source_documents=True)
#query = "what is the impact of JUSE on TQM?"
#query = "what is the purpouse of life?"
#query = "what does demming like for lunch"
#query = "is quality like a human"
#query = "what is the fourth rule of TQM"
query = "what is Total Quality Management (TQM)"
result = qa({"query": query})

In [19]:
result['result']

' Total Quality Management (TQM) is a system of management that seeks to continuously improve the quality of products and services through the involvement of all levels of employees in the organization. It is achieved by setting quality standards, measuring performance against those standards, and taking corrective actions to ensure that the standards are met.'

In [20]:
result['source_documents']

[Document(page_content='1) AGREE: Advisory Group on Reliability of Electronic Equipment 2) AOTS: Association for Overseas Technical Scholarship 3) APO: Asian Productivity Organization 4) ASA: American Standards Association 5) ASQC: American Society for Quality Control 6) BS: British Standards 7) CCS: Civil Communication Section 8) COSCO: Control System Committee 9) EOQC: European Organization for Quality Control 10) EOQ: European Organization for Quality 11) EFQM: European Foundation for Quality Management 12) FMES: Japan Federation of Managerial Engineering Societies\n\n524\n\nA Parallel Chronology of Prof. Kaoru Ishikawa’s Life and TQC in Japan', metadata={'source': '/content/gdrive/MyDrive/Projects/Digital_Demming/src_content/ParallelChronology_Ver2_150717.pdf'}),
 Document(page_content='Medal. (ASQC)\n\n\uf09f The Pharmaceutical Manufacturing and Quality Control (GMP) Seminar was held. (Sep., JUSE)\n\n\uf09f The Japan Prefabricated Construction\n\nSuppliers and Manufacturers Associ

In [21]:
# Create a list to stuff section results into 
introduction = []

#append result  to the array
introduction.append(result)

#print the array
print(introduction)

[{'query': 'what is Total Quality Management (TQM)', 'result': ' Total Quality Management (TQM) is a system of management that seeks to continuously improve the quality of products and services through the involvement of all levels of employees in the organization. It is achieved by setting quality standards, measuring performance against those standards, and taking corrective actions to ensure that the standards are met.', 'source_documents': [Document(page_content='1) AGREE: Advisory Group on Reliability of Electronic Equipment 2) AOTS: Association for Overseas Technical Scholarship 3) APO: Asian Productivity Organization 4) ASA: American Standards Association 5) ASQC: American Society for Quality Control 6) BS: British Standards 7) CCS: Civil Communication Section 8) COSCO: Control System Committee 9) EOQC: European Organization for Quality Control 10) EOQ: European Organization for Quality 11) EFQM: European Foundation for Quality Management 12) FMES: Japan Federation of Managerial

In [22]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch, return_source_documents=True)
#query = "what is the impact of JUSE on TQM?"
#query = "what is the purpouse of life?"
#query = "what does demming like for lunch"
#query = "is quality like a human"
#query = "what is the fourth rule of TQM"
query = "Write a detailed introduction of the significance of TQM in the modern business world"
result = qa({"query": query})

In [23]:
result['result']

"\n\nTotal Quality Management (TQM) is a management approach that seeks to improve the quality of products and services by focusing on customer satisfaction, continuous improvement, and employee involvement. TQM is a process-oriented approach to management, which seeks to ensure that all aspects of the organization are held to the highest standards of quality and efficiency. TQM is also a customer-centric approach, which means that it puts the customer's needs and wants at the center of all decisions and actions. By doing so, TQM encourages organizations to think about their customers first and take proactive steps to ensure that their products and services are meeting the customer's needs. By continuously improving and innovating, TQM helps organizations to stay competitive and successful in the marketplace. Additionally, by emphasizing employee involvement and empowerment, TQM encourages employees to take responsibility for their own job performance and to come up with creative solut

In [24]:
#append result  to the array
introduction.append(result)

#print the array
print(introduction)

[{'query': 'what is Total Quality Management (TQM)', 'result': ' Total Quality Management (TQM) is a system of management that seeks to continuously improve the quality of products and services through the involvement of all levels of employees in the organization. It is achieved by setting quality standards, measuring performance against those standards, and taking corrective actions to ensure that the standards are met.', 'source_documents': [Document(page_content='1) AGREE: Advisory Group on Reliability of Electronic Equipment 2) AOTS: Association for Overseas Technical Scholarship 3) APO: Asian Productivity Organization 4) ASA: American Standards Association 5) ASQC: American Society for Quality Control 6) BS: British Standards 7) CCS: Civil Communication Section 8) COSCO: Control System Committee 9) EOQC: European Organization for Quality Control 10) EOQ: European Organization for Quality 11) EFQM: European Foundation for Quality Management 12) FMES: Japan Federation of Managerial

In [25]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch, return_source_documents=True)
#query = "what is the impact of JUSE on TQM?"
#query = "what is the purpouse of life?"
#query = "what does demming like for lunch"
#query = "is quality like a human"
#query = "what is the fourth rule of TQM"
#query = "Write a detailed introduction of the significance of TQM in the modern business world"
query = "Introduction to Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)"
result = qa({"query": query})

In [26]:
result['result']

' Kaoru Ishikawa worked as a lecturer of the Introductory Course for Design of Experiment in the Department of Chemical Engineering at the University of Tokyo. He also became a member of the ISO/TC 102 Japan Committee, the chair of the Sampling Subcommittee, the chair of the editorial board of Genba To QC-magazine, the chair of the Editorial Committee of the JUSE English Report, the vice-chair of the Reliability and Maintainability Symposium Organizing Committee, and the chair of the Organizing Committee of the JUSE Course for Purchasing. He was also involved in the QC Seminar for Managers and the QC Seminar for Supervisors (Central Japan Industries Association), the Inspection Seminar (JSA), the QC Correspondence Basic Course for Foremen (JUSE), the QC Circle Hokkaido Regional Chapter (QC Circle Headquarters), the Japanese Society for Quality Control foundation meeting (Nov.), the Sensory Test Overseas Research Team (May, JUSE), the Standardization Mission Team (headed by Hidehiko Azu

In [27]:
#append result  to the array
introduction.append(result)

#print the array
print(introduction)

[{'query': 'what is Total Quality Management (TQM)', 'result': ' Total Quality Management (TQM) is a system of management that seeks to continuously improve the quality of products and services through the involvement of all levels of employees in the organization. It is achieved by setting quality standards, measuring performance against those standards, and taking corrective actions to ensure that the standards are met.', 'source_documents': [Document(page_content='1) AGREE: Advisory Group on Reliability of Electronic Equipment 2) AOTS: Association for Overseas Technical Scholarship 3) APO: Asian Productivity Organization 4) ASA: American Standards Association 5) ASQC: American Society for Quality Control 6) BS: British Standards 7) CCS: Civil Communication Section 8) COSCO: Control System Committee 9) EOQC: European Organization for Quality Control 10) EOQ: European Organization for Quality 11) EFQM: European Foundation for Quality Management 12) FMES: Japan Federation of Managerial

In [51]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch, return_source_documents=True)
#query = "what is the impact of JUSE on TQM?"
#query = "what is the purpouse of life?"
#query = "what does demming like for lunch"
#query = "is quality like a human"
#query = "what is the fourth rule of TQM"
#query = "Write a detailed introduction of the significance of TQM in the modern business world"
#query = "Introduction to Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)"
query = " Write a Thesis Statement regarding TQM, Kaoru Ishikawa, Japanese Union of Scientists and engineers."
result = qa({"query": query})

In [52]:
result['result']

" Thesis Statement: Kaoru Ishikawa's life and work in TQC, which was strongly supported by the Japanese Union of Scientists and Engineers, had a major impact on the development of Total Quality Management in Japan."

In [30]:
#append result  to the array
introduction.append(result)

#print the array
print(introduction)

[{'query': 'what is Total Quality Management (TQM)', 'result': ' Total Quality Management (TQM) is a system of management that seeks to continuously improve the quality of products and services through the involvement of all levels of employees in the organization. It is achieved by setting quality standards, measuring performance against those standards, and taking corrective actions to ensure that the standards are met.', 'source_documents': [Document(page_content='1) AGREE: Advisory Group on Reliability of Electronic Equipment 2) AOTS: Association for Overseas Technical Scholarship 3) APO: Asian Productivity Organization 4) ASA: American Standards Association 5) ASQC: American Society for Quality Control 6) BS: British Standards 7) CCS: Civil Communication Section 8) COSCO: Control System Committee 9) EOQC: European Organization for Quality Control 10) EOQ: European Organization for Quality 11) EFQM: European Foundation for Quality Management 12) FMES: Japan Federation of Managerial

Let's see if we can output this to markdown

In [31]:
concatenated_data = [str(item) for item in introduction]
for entry in concatenated_data:
    print("Processing entry: ")
    messages = [
        SystemMessage(content="You are an AI that takes dictionary data and translates it into well formatted markdown. You create sections and subsections to communicate as much information as possible"),
        HumanMessage(content=entry)
    ]
    result = chat(messages)

    print("Result: ", result)

Processing entry: 
Result:  content='# Total Quality Management (TQM)\n\nTotal Quality Management (TQM) is a system of management that seeks to continuously improve the quality of products and services through the involvement of all levels of employees in the organization. It is achieved by setting quality standards, measuring performance against those standards, and taking corrective actions to ensure that the standards are met.\n\n## Source Documents\n\n- AGREE: Advisory Group on Reliability of Electronic Equipment\n- AOTS: Association for Overseas Technical Scholarship\n- APO: Asian Productivity Organization\n- ASA: American Standards Association\n- ASQC: American Society for Quality Control\n- BS: British Standards\n- CCS: Civil Communication Section\n- COSCO: Control System Committee\n- EOQC: European Organization for Quality Control\n- EOQ: European Organization for Quality\n- EFQM: European Foundation for Quality Management\n- FMES: Japan Federation of Managerial Engineering Soc

In [33]:

  messages = [
      SystemMessage(content="You are an assistant helping me research a paper"),
      HumanMessage(content="return an outline for the book - The History and Evolution of Total Quality Management (TQM): The Contributions of Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE) ")
  ]
  result = chat(messages)

  print("Result: ", result)

Result:  content="I. Introduction\n- Brief overview of Total Quality Management (TQM)\n- Importance of studying the history and evolution of TQM\n- Thesis statement\n\nII. The Origins of TQM\n- Early quality control practices in manufacturing\n- The role of W. Edwards Deming in the development of TQM\n- The impact of World War II on TQM\n\nIII. Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)\n- Ishikawa's background and contributions to TQM\n- The formation and goals of JUSE\n- The development of Ishikawa's quality control tools\n\nIV. The Spread of TQM\n- The influence of JUSE on TQM in Japan\n- The adoption of TQM by American companies\n- The impact of TQM on global business practices\n\nV. Criticisms and Challenges of TQM\n- Criticisms of TQM's effectiveness\n- Challenges in implementing TQM in organizations\n- The future of TQM\n\nVI. Conclusion\n- Summary of key points\n- Implications for future research and practice." additional_kwargs={} example=False


In [56]:
#from langchain.prompts import PromptTemplate
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer as if you are a management consultant. Use specific dates and references. Use bullet points and tabels where appropriate. Reference source documents:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=docsearch.as_retriever(), chain_type_kwargs=chain_type_kwargs)
query = "Who won the Deming prize?"
qa.run(query)

'\n\nThe 1st Deming Prize Ceremony was held at the same time as the 1st QC Conference in September 1952. The first winners of the Deming Prize were: \n\n• Deming Prize for Individuals: Motosaburo Masuyama \n• Deming Application Prize: \n    - Showa Denko \n    - Tanabe Pharmaceuticals\n    - Fuji Iron & Steel \n    - Yawata Iron & Steel'

In [59]:
query = "when was the last deming prize awarded"
qa.run(query)

"\n\nThe last Deming Prize was awarded in 1986. According to the Parallel Chronology of Prof. Kaoru Ishikawa's Life and TQC in Japan, the 41th Quality Control Conference (QCS) was held in 1986 and the 15th Anniversary Party of the establishment of the Deming Prize was held (Nov, JUSE). During this event, recipients of the Deming Prize were awarded."

In [60]:
query = "who invented QC circle?"
qa.run(query)

'\n\nThe invention of QC circles is often attributed to Professor Kaoru Ishikawa, a Japanese professor at the University of Tokyo during the 1960s and 1970s. He is credited with developing the concept of QC circles and promoting them across Japan and the world. \n\nIn the early 1960s, Prof. Ishikawa and other quality control researchers visited the United States and observed the operations of Lockheed Aircraft Corporation, where they saw the operation of QC circles. This inspired Prof. Ishikawa to introduce the concept of QC circles to Japan, and he started promoting them in 1962. \n\nIn 1964, the Federation of Pharmaceutical Manufacturers’ Association of Japan (JGMP) also issued a guideline for the introduction of QC circles in the production of medical products. This encouraged the spread of QC circles in Japan. \n\nIn the following years, more countries began to adopt QC circles, with the USSR Standardization Delegation visiting Japan in 1967 and the China Standardization and Qualit

In [61]:
query = "when was the QC Circle Seminar for Top Management held"
qa.run(query)

"\n\nThe QC Circle Seminar for Top Management was held in 1973. It was organized by Prof. Kaoru Ishikawa, and was organized by the JUSE (Japan Union of Scientists and Engineers). Major topics discussed included Quality Control as a tool to overcome the economic depression, Our Company's Company-wide Quality Control, New Technology Development/New Product Planning and Quality Control, and Industrial Product Quality Improvement Training Course Follow-up Seminar for Participants from Developing Countries."

## II. Early History of Quality Management
- Brief description of quality management before TQM
- Introduction to foundational concepts: Inspection, Quality Control, and Quality Assurance
- The rise of statistical process control and its significance

## III. The Birth and Evolution of TQM
- Introduction to the concept of TQM
- Overview of the principles and practices of TQM
- The role of TQM in post-World War II Japan's industrial comeback
- Adoption and evolution of TQM in the United States and around the world

## IV. The Role of Kaoru Ishikawa in the Development of TQM
- Introduction to Ishikawa: his background and career
- Ishikawa's contribution to the development of Quality Circles
- The development and importance of the Ishikawa (cause-and-effect) Diagram
- Ishikawa's philosophical contribution to TQM: the 'next process is your customer'


## V. The Role of the Japanese Union of Scientists and Engineers (JUSE)
- Background of JUSE and its role in the development of quality management in Japan
- Introduction of the Deming Prize by JUSE and its significance
- JUSE's role in promoting TQM and its principles in Japan and globally


## VI. The Impact of TQM, Ishikawa, and JUSE on Contemporary Business Practices
- Analysis of the role TQM plays in today's business and management practices
- The legacy of Ishikawa's methods and tools in current quality management
- Influence of JUSE's principles in contemporary quality initiatives and awards


## VII. Conclusion
- Summary of the importance and influence of TQM, Ishikawa, and JUSE in the field of quality management
- Restatement of Thesis
- Future implications of the principles and practices of TQM, Ishikawa, and JUSE in the global business landscape